# New implementation

What can the bot do?:
1. Respond to user questions.
2. Get links from google and combines it with response and returns to the user.
3. Store the whole conversation in a csv file.
4. Retrieve answers from a database that are not found in the internet.
5. Interpret code and suggest edits.
6. Send an email to a concerened person when the user wants to escalate.



In [ ]:
!pip3 install openai langchain pinecone-client tiktoken numpy pandas flask firebase-admin firebase-auth streamlit pytest python-dotenv pylint
!pip install promptlayer
!pip install openai
!pip install openai >> null
!pip install beautifulsoup4
!pip install google
!pip install beautifulsoup4

In [16]:
%%capture
import json
import openai
import time
import os
import sys
from openai import OpenAI

In [17]:
#set environment variable. (include your api keys)
os.environ['OPENAI_API_KEY'] = 'your key'    #you can find through this link: https://platform.openai.com/api-keys

#not needed
# os.environ['PINECNE_API_KEY'] = 'your key'
# os.environ['PROMPTLAYER_API_KEY'] = 'your key'

In [18]:
key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(
    organization= "your key",      #you can find through this link: https://platform.openai.com/account/organization
    api_key = key
)

In [19]:
# Upload files like FAQ,
# Upload a file with an "assistants" purpose  (can upload csv, pdf, doc file)
file = client.files.create(
    file=open("question_answers.csv", "rb"), purpose='assistants'
)
file_id = file.id
# print(file.id)

In [20]:
# create assistant, give it access to file. And print out assistant id.
prompt = """ You are PyVizPro, a data visualization expert with a strong command of Python. You've spent years honing your data visualization skills, and you're ready to help others understand how to create stunning visuals from their data. You use examples from well-known data visualization libraries like Matplotlib, Seaborn, Plotly and provide code snippets and resources to help illustrate your points.
Your language must be easy to grasp for someone new to data visualization in Python, and you'll use a mix of relatable analogies and everyday language to make your explanations engaging.
If you encounter a question you can't answer, don't make things up; instead, ask for more details to provide the best answer.
If the user asks questions not related to data visualization and python, you must not answer that question and inform the user that "Please note, I specialize in Python data visualization. Questions outside this realm is beyond my expertise."
"""
assistant = client.beta.assistants.create(
    name="PyVizPro",
    instructions = prompt,
    model="gpt-4-1106-preview",
    tools=[
            {"type": "retrieval"},
            {"type": "code_interpreter"},     #we can also add a type: function and then define a function as well.
          ],
    file_ids=[file_id]        #if no files to read from, just remove this file ids.
)

# print(assistant)

#Helper Functions

In [21]:
#helper function: send email
#html format available.
'''
https://github.com/visheshdvivedi/How-To-Send-Email-In-Python-Using-Smtplib-Tutorial/blob/main/html%20mail.py
'''
import smtplib
import getpass
def send_email(from_email, to_email, subject, body_text):

    HOST = "smtp-mail.outlook.com"
    PORT = 587

    FROM_EMAIL = from_email
    TO_EMAIL = to_email
    PASSWORD = getpass.getpass("Enter your email password: ")

    # MESSAGE = """Subject: this is new test email \n
    # Hello i am testing this email function and this is body text. sajan"""

    MESSAGE = f"Subject: {subject}\n\n{body_text}"

    try:
            smtp = smtplib.SMTP(HOST, PORT)
            status_code, response = smtp.ehlo()
            print(f"[*] Echoing the server: {status_code} {response}")

            status_code, response = smtp.starttls()
            print(f"[*] Starting TLS connection: {status_code} {response}")

            status_code, response = smtp.login(FROM_EMAIL, PASSWORD)
            print(f"[*] Logging in: {status_code} {response}")

            smtp.sendmail(FROM_EMAIL, TO_EMAIL, MESSAGE)
            smtp.quit()
            return True

    except Exception as e:
            print(f"An error occurred: {e}")
            return False


# FROM_EMAIL = "rsajanacharya20@outlook.com"
# TO_EMAIL = "sajanacharya20@gmail.com"
# subject = "test subject"
# body_text = "Hello i am testing this email function and this is body text.sajan"
# send_email(FROM_EMAIL, TO_EMAIL, subject, body_text)

In [30]:
#store questions and answers in csv file.
import csv
def store_answers_in_csv(questions_response_dict):

  # Define the file name for the CSV
  file_name = 'question_answers.csv'

  # Writing question-answer pairs to a CSV file
  #append to the file: a |  create new file and write: w
  with open(file_name, mode='a', newline='') as file:
      writer = csv.writer(file)

      # Writing header (optional)
      writer.writerow(['Question', 'Answer'])

      # Writing question-answer pairs
      for question, answers in questions_response_dict.items():
          writer.writerow([question, answers])
  print(f"Question-answer pairs have been written to '{file_name}'.")


# Your dictionary with questions and lists of answers:
#Sample Input->  questions_response_dict = {question1: answer1, question2: answer2}

In [23]:
def store_in_database(question, answer):
  #TO DO add it in the database
  pass

In [24]:
def escalate():
  #add what we need to do if the user wants to escalate the issue to human

  #TO DO
  # 1. get the email, password and send the conversation as required body text. All conversation currently in csv
  # 2. Another approach: we can summarize the conversation(using other llm which doesn't cost much) and then send that as body text as well.

  # we can send email like this.
  # sent = send_email('from_email',destination_email, subject,text_body)
  email_sent = send_email('rsajanacharya20@outlook.com', 'sajanacharya20@gmail.com', "this is a test email", "Hello i am testing this email function and this is body text. sajam")
  if email_sent:
    return "Email has been sent"
  else:
    return "There has been some error sending email"


In [32]:
#bold prompt
def bold_prompt(prompt):
    bold = "\033[1m"
    reset = "\033[0m"
    sys.stdout.write(bold + prompt + reset)
    sys.stdout.flush()

def bold_print(text):
    return f"\033[1m{text}\033[0m"


In [33]:
# bold_prompt("Did the following response address your question? REPLY YES or NO: ")
# ask_user_satisfaction = input()

# normal = input("This is normal type")
# print(normal)

In [34]:
""" This function takes gpt's answers as parameter and searches on the web to get relevant website links to
    be returned to the users for references and further studies.
    We use google api to conduct the search
"""
# creating a set of websites we want to omit
websites_to_omit = {"https://study.com"}

def get_links(gpt_response):

    # a list to store the links
    resources_links = []
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google' found")
        return []

    # use sijans summary function that will summarize gpts response which can then be used
    # to search the web

    # what_to_search = summarize_Llama(gpt_response)


    # num sets no. of result, start sets where to start retreiving result from
    # stop sets last number of result to retreive
    # pause is to pause the http requests
    for link in search(gpt_response, num=10, stop=15, pause=5):
        # print(link)
        curr_link =  link.split(".com")
        if f"{curr_link[0]}.com" not in websites_to_omit:
            resources_links.append(link)

    return resources_links

#Main Function



In [ ]:
from openai.types.beta import assistant_create_params
#TO DO: divide the code into modular programming.  need to work more on error handling
def start_chatbot():
  thread = client.beta.threads.create()
  keep_asking = True
  message_response_dict = {}
  follow_up_required = False

  while keep_asking:
    if not follow_up_required:   #for Automated Follow-up System
      user_question = input("How can i help you? ")

    else:
      #follow up is required. User can either provide more details or escalate(call human)
      user_question = input("Please provide more details or type 'escalate' to escalate the issue:")
      follow_up_required = False

      # TO DO: If a new email has content NO, then the bot should forward the email to the designated help desk ticket email
      if user_question.lower() == "escalate":
        #if user wants to escalate, the bot will store the questions and response to csv file and then we can either send it to email or something.
        #just for initial phase to store the question and response to csv file.
        store_answers_in_csv(message_response_dict)
        message_response_dict = {}             #clearing for next query
        escalate()
        keep_asking = False
        break

    # sends the user's message to the OpenAI thread
    message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = user_question
    )

    #Running the Assistant:
    run = client.beta.threads.runs.create(
        thread_id = thread.id,
        assistant_id = assistant.id
    )
    # checks the status of the assistant. While the status is not complete, the
    try_count = 0
    print("loading", end = "") #to show that pyvizpro is searching for answer.
    while run.status not in ["completed", "failed", "requires_action"]:
        print("...", end = "")
        if try_count > 0:
            time.sleep(5)      #decrease the amount of times we check the run status. Wait for the gpt to generate an answer.

        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        try_count += 1

    if run.status == "Failed":
      #do something
      print("An errorr has occured with the api")
      break


    #Getting the Assistant's Response:
    complete_response = client.beta.threads.messages.list(thread_id=thread.id)

    # find the latest response received from the assistant within a list of messages
    latest_response_received = next(
        (msg for msg in reversed(complete_response.data) if msg.run_id == run.id and msg.role == "assistant"),
        None
    )

    #just getting the message
    if latest_response_received:
        assistant_message = latest_response_received.content[0].text.value
    else:
        print("No message found from the assistant in this run.")
        continue
    print("\n")
    print(f"{bold_print('User')}: {user_question}")
    print("===========================")
    if "beyond my expertise" not in assistant_message:
      #gets resources links everytime unless the question is out of scope.
      resources_links = get_links(assistant_message)
      if resources_links:
        links_string = "\n".join(resources_links)
        bolded_line = bold_print("Here are some resources you can explore further:")
        print(f"{bold_print('Assistant')}: {assistant_message}\n\n{bolded_line}\n{links_string} ")
        assistant_message_with_links = f'{assistant_message}\n{links_string}'
    else:
      print(f"{bold_print('Assistant')}: {assistant_message}")   #if the message if out of scope. We can directly just put 'Please note, I specialize in Python data visualization. Questions outside this realm is beyond my expertise.'

    #initial approach to store question and answer for a single user.
    message_response_dict[user_question] = assistant_message_with_links

    print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    print("\n")

    #this is the function to bold the prompt and get the response
    bold_prompt("Did the following response address your question? REPLY YES or NO: ")
    ask_user_satisfaction = input()

    if ask_user_satisfaction.upper() == "YES":
      # # can add the question and response to the database for next time ml integration.
      # store_in_database(user_question, assistant_message)
      ask_user_end = input("Do you want to end this chat? (Y/N) ")
      if ask_user_end.lower() == "y":
        #stores the convo to the csv file
        store_answers_in_csv(message_response_dict)
        print("Hope you had your questions solved. Have a great day")
        break
    elif ask_user_satisfaction.upper() == "NO":
      follow_up_required = True
      #follow up is required and then they can ask whether user wants to talk to human or not. In next iteration followup will be executed
    print("\n\n")
  return bold_print("Hope you had your questions solved.Have a great day")

start_chatbot()